# Code for full simulation

In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt

## Financial Pillar Class Construction

In [2]:
class Income:
    def __init__(self, base_salary, salary_increase_rate):
        self.current_salary = base_salary  # Track salary dynamically
        self.salary_increase_rate = salary_increase_rate / 100  # Convert to decimal

    def yearly_pay_adjustment(self):
        """Increase salary by the raise percentage each year."""
        self.current_salary *= (1 + self.salary_increase_rate)

    def get_salary(self):
        """Return the current salary (used in Simulation)."""
        return self.current_salary

In [3]:
class PersonalInvestmentAccount:
    def __init__(self, balance, annual_contribution, investment_growth_rate):
        self.balance = balance  # Initial personal investment balance
        self.annual_contribution = annual_contribution  # Amount invested each year
        self.investment_growth_rate = investment_growth_rate / 100  # Convert percentage to decimal

    def contribute(self):
        """Add the annual contribution to personal investments."""
        self.balance += self.annual_contribution

    def grow(self):
        """Apply investment growth rate to the personal investments."""
        self.balance *= (1 + self.investment_growth_rate)


In [4]:
class RothIRA:
    def __init__(self, initial_balance, annual_contribution, investment_growth_rate):
        self.balance = initial_balance  # Starting balance of Roth IRA
        self.annual_contribution = annual_contribution  # Fixed yearly contribution
        self.investment_growth_rate = investment_growth_rate / 100  # Convert percentage to decimal

    def contribute(self):
        """Add annual Roth IRA contribution (subject to IRS limits, not enforced here)."""
        self.balance += self.annual_contribution

    def grow(self):
        """Apply investment growth to Roth IRA balance."""
        self.balance *= (1 + self.investment_growth_rate)


In [5]:
class K401:
    def __init__(self, initial_balance, personal_contribution_rate, employer_match_rate, investment_growth_rate):
        self.balance = initial_balance  # Starting 401(k) balance
        self.personal_contribution_rate = personal_contribution_rate / 100  # Convert percentage to decimal
        self.employer_match_rate = employer_match_rate / 100  # Convert percentage to decimal
        self.investment_growth_rate = investment_growth_rate / 100  # Convert percentage to decimal

    def contribute(self, annual_income):
        """Calculate and add contributions to 401(k)."""
        personal_401k = annual_income * self.personal_contribution_rate
        employer_401k = annual_income * self.employer_match_rate
        self.balance += personal_401k + employer_401k

    def grow(self):
        """Apply investment growth to 401(k)."""
        self.balance *= (1 + self.investment_growth_rate)


In [6]:
class Expenses:
    def __init__(self, food, housing, transportation, other, annual_increase_rate):
        # Track current year's expenses dynamically
        self.current_food = food
        self.current_housing = housing
        self.current_transportation = transportation
        self.current_other = other
        self.annual_increase_rate = annual_increase_rate / 100  # Convert percentage to decimal

    def yearly_expense_adjustment(self):
        """Increase each expense category by the annual increase rate."""
        self.current_food *= (1 + self.annual_increase_rate)
        self.current_housing *= (1 + self.annual_increase_rate)
        self.current_transportation *= (1 + self.annual_increase_rate)
        self.current_other *= (1 + self.annual_increase_rate)

    def get_expenses(self):
        """Return the current year's expenses as a dictionary."""
        return {
            'food': self.current_food,
            'housing': self.current_housing,
            'transportation': self.current_transportation,
            'other': self.current_other
        }


## Life Events

In [7]:
class LoseJob:
    def __init__(self, year, income):
        self.year = year
        self.income = income  # Store the reference to the income object

    def apply(self):
        self.income.current_salary = 0  # Now it correctly modifies the income object

## Build Simulation Class

In [8]:
class Simulation:
    def __init__(self, income, personal_investment_account, k_401, roth_ira, expenses, losejob, years):
        self.income = income
        self.personal_investment_account = personal_investment_account
        self.k_401 = k_401
        self.roth_ira = roth_ira
        self.expenses = expenses
        self.years = years
        self.losejob = losejob

    def run(self):
        results = []
        for year in range(self.years):
            # Apply Job Loss Before Calculating Income
            if year == self.losejob.year:
                self.losejob.apply()

            # Collect annual income and expenses
            annual_income = self.income.get_salary()  # Get current salary
            expenses_breakdown = self.expenses.get_expenses()

            # Increase salary and expenses for next year
            self.income.yearly_pay_adjustment()
            self.expenses.yearly_expense_adjustment()

            # Calculate 401(k) Contributions
            personal_401k = annual_income * (self.k_401.personal_contribution_rate)
            employer_401k = annual_income * (self.k_401.employer_match_rate)

            # Contribute to 401(k), Roth IRA, and Personal Investment Account
            self.k_401.contribute(annual_income)
            self.roth_ira.contribute()
            self.personal_investment_account.contribute()

            # Grow investments
            self.k_401.grow()
            self.roth_ira.grow()
            self.personal_investment_account.grow()

            # Compute net cash flow
            total_expenses = sum(expenses_breakdown.values())

            results.append({
                'year': year + 1,
                'income': annual_income,
                'total_expenses': total_expenses,
                'expenses': expenses_breakdown,
                'personal_investment_account': self.personal_investment_account.balance,
                '401k_balance': self.k_401.balance,
                'roth_ira_balance': self.roth_ira.balance,
            })

        return pd.DataFrame(results)

    def run_simulation(self):
        return self.run()


## Run Simulation

In [9]:
def load_inputs(file_path = "inputs.yaml"):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
        return config

In [10]:
example_yaml = load_inputs()
#example_yaml

In [11]:
def run_test():
    income = Income(**example_yaml['income'])
    personal_investment_account = PersonalInvestmentAccount(**example_yaml["personal_investment_account"])

    # Create separate 401(k) and Roth IRA objects
    k_401 = K401(**example_yaml["k_401"])
    roth_ira = RothIRA(**example_yaml["roth_ira"])

    expenses = Expenses(**example_yaml['expenses'])
    losejob = LoseJob(example_yaml['life_events']['job_loss']['year'], income)

    sim = Simulation(income, personal_investment_account, k_401, roth_ira, expenses, losejob, example_yaml['years'])
    simulation_output = sim.run_simulation()
    return simulation_output

# Run the simulation
simulation_output = run_test()
simulation_output


,year,income,total_expenses,expenses,personal_investment_account,401k_balance,roth_ira_balance
0,1,120000.000000,43000.000000,"{'food': 8000, 'housing': 24000, 'transportati...",5350.000000,7.404400e+04,6420.000000
1,2,123600.000000,44075.000000,"{'food': 8200.0, 'housing': 24599.999999999996...",11074.500000,1.003874e+05,13289.400000
2,3,127308.000000,45176.875000,"{'food': 8405.0, 'housing': 25214.999999999993...",17199.715000,1.292096e+05,20639.658000
3,4,131127.240000,46306.296875,"{'food': 8615.125, 'housing': 25845.3749999999...",23753.695050,1.607033e+05,28504.434060
4,5,135061.057200,47463.954297,"{'food': 8830.503125, 'housing': 26491.5093749...",30766.453704,1.950750e+05,36919.744444
5,6,139112.888916,48650.553154,"{'food': 9051.265703124998, 'housing': 27153.7...",38270.105463,2.325464e+05,45924.126555
6,7,143286.275583,49866.816983,"{'food': 9277.547345703122, 'housing': 27832.6...",46299.012845,2.733552e+05,55558.815414
7,8,147584.863851,51113.487408,"{'food': 9509.486029345699, 'housing': 28528.4...",54889.943744,3.177566e+05,65867.932493
8,9,152012.409767,52391.324593,"{'food': 9747.22318007934, 'housing': 29241.66...",64082.239806,3.660241e+05,76898.687768
9,10,156572.782060,53701.107708,"{'food': 9990.903759581322, 'housing': 29972.7...",73917.996593,4.184511e+05,88701.595911
